# Fine-tuning VistralPoem5 with Axolotl

In [ ]:
import huggingface_hub
authorized_token = "Type your authorization token here"
huggingface_hub.login(token=authorized_token)

### Install Libraries and Import Needed

In [ ]:
!pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121
!pip install kaleido cohere openai
!pip install datasets -U
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install attrs -U
!pip install accelerate -U

In [ ]:
!git clone -q https://github.com/OpenAccess-AI-Collective/axolotl
%cd axolotl
!pip install packaging huggingface_hub torch==2.1.2 --progress-bar off
!pip install -e '.[flash-attn,deepspeed]'

In [ ]:
import yaml
from huggingface_hub import HfApi
from google.colab import userdata

### Setting Configs

In [ ]:
yaml_string = """
base_model: Viet-Mistral/Vistral-7B-Chat
model_type: MistralForCausalLM
tokenizer_type: AutoTokenizer
is_mistral_derived_model: true

load_in_8bit: false
load_in_4bit: true
strict: false

datasets:
  - path: pphuc25/poem-5-words-vietnamese
    type:
      system_prompt: "Bạn là một nhà thơ chuyên nghiệp, nhiệm vụ của bạn là chuyển bài văn này thành 1 bài thơ 5 chữ từ khoảng 1 đến 3 khổ"
      field_system: system
      field_instruction: input
      field_output: output
      format: "[INST] <<SYS>>\n{instruction}\n<</SYS>>\n\n{input} [/INST] "

dataset_prepared_path:
val_set_size: 0.05
output_dir: ./qlora-out

adapter: qlora
lora_model_dir:

sequence_len: 1096
sample_packing: true
pad_to_sequence_len: true

lora_r: 32
lora_alpha: 16
lora_dropout: 0.1
lora_target_modules:
lora_target_linear: true
lora_fan_in_fan_out:

wandb_project:
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:


mlflow_experiment_name: colab-example

gradient_accumulation_steps: 4
micro_batch_size: 16
num_epochs: 2
max_steps: 100
optimizer: adamw_bnb_8bit
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: auto
fp16:
tf32: false

gradient_checkpointing: true
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention:
flash_attention: true

warmup_steps: 10
evals_per_epoch:
saves_per_epoch:
debug:
deepspeed:
weight_decay: 0.0
fsdp:
fsdp_config:
special_tokens:
  bos_token: "<s>"
  eos_token: "</s>"
  unk_token: "<unk>"
"""

# Convert the YAML string to a Python dictionary
yaml_dict = yaml.safe_load(yaml_string)

# Specify your file path
yaml_file = 'config.yaml'

# Write the YAML file
with open(yaml_file, 'w') as file:
    yaml.dump(yaml_dict, file)

### Training Model

In [ ]:
!accelerate launch -m axolotl.cli.train config.yaml

### Merge Model

In [ ]:
!python3 -m axolotl.cli.merge_lora config.yaml --lora_model_dir="./qlora-out"

### Push model

In [ ]:
new_model = "Type your model name here"

# HF_TOKEN defined in the secrets tab in Google Colab
api = HfApi()

# Upload merge folder
api.create_repo(
    repo_id=new_model,
    repo_type="model",
    exist_ok=True,
)
api.upload_folder(
    repo_id=new_model,
    folder_path="qlora-out/merged",
)